In [1]:
import os 

In [2]:
%pwd

'e:\\2025\\Pojects\\Chest-Cancer-Classification-using-MLflow-DVC\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\2025\\Pojects\\Chest-Cancer-Classification-using-MLflow-DVC'

In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    update_base_model_path: Path
    training_data: Path
    validation_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [13]:
from CnnClassifier.constants import *
from CnnClassifier.utils.common import read_yaml,create_directories
import tensorflow as tf

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data",'train')
        validation_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data",'valid')
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            update_base_model_path=Path(prepare_base_model.update_base_model_path),
            training_data=Path(training_data),
            validation_data =Path(validation_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import tensorflow as tf

In [25]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.update_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.validation_data,
            subset="validation",
            shuffle=False,
            class_mode='categorical',
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            class_mode='categorical',
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        self.steps_per_epoch = max(1, self.train_generator.samples // self.train_generator.batch_size)
        self.validation_steps = max(1, self.valid_generator.samples // self.valid_generator.batch_size)

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [26]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-10-30 08:20:23,154 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-10-30 08:20:23,163 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-10-30 08:20:23,166 : INFO : common : created directory at : artifacts]
[2025-10-30 08:20:23,169 : INFO : common : created directory at : artifacts\training]
Found 13 images belonging to 4 classes.
Found 491 images belonging to 4 classes.
30/30 [==============================] - 216s 7s/step - loss: 23.5184 - accuracy: 0.2463 - val_loss: 23.9452 - val_accuracy: 0.1538


e:\2025\Pojects\Chest-Cancer-Classification-using-MLflow-DVC\.venv_tf\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
